In [1]:
import numpy as np
from utils.download_from_GCP import download_table_to_local_as_one_file
import pandas as pd
import os
from sklearn.metrics import ndcg_score, roc_auc_score
from tqdm import tqdm
from multiprocessing import Pool
import multiprocessing as mp
import pathlib
import xlearn as xl

In [2]:
LANGS = [
#     "Hindi",
    # "Tamil",
#     "Telugu",
#     "Kannada",
    "Punjabi",
# #     "Odia",
    "Bengali",
# #     "Marathi",
    "Malayalam",
#     "Gujarati",
]
# DAYS_OF_DATA_CONSIDERED = 7
rating_def_dict = {
        "vplay": "is_vp_succ",
        "like": "is_like",
# #         "share": "is_share",
# #         "fav": "is_fav",
        "vplay_skip": "is_vp_skip",
        "vplay2": "is_vp_succ2",
    }

# user_post_ref_mapping_file_path = "./train_test_data_models/Bengali/is_vp_succ/user_post_ffm_mapping.csv"
# predicted_results_path = "./train_test_data_models/Bengali/is_vp_succ/predicted_results/predicted_scores.txt"
# test_file_path = "./train_test_data_models/Bengali/is_vp_succ/test.txt"
# no_ref_user_post_ref_mapping_file_path = "./train_test_data_models/Bengali/is_vp_succ/user_post_ffm_mapping.csv"
# no_ref_predicted_results_path = "./train_test_data_models/Bengali/is_vp_succ/predicted_results/predicted_scores.txt"
# no_ref_test_file_path = "./train_test_data_models/Bengali/is_vp_succ/test.txt"

In [3]:
referrer_level_df = None
for lang in LANGS:
    for _, rating_def in rating_def_dict.items():
        print(f"Moving to {lang}-{rating_def}")
        user_post_ref_mapping_file_path = f"./train_test_data_models/{lang}/{rating_def}/user_post_ffm_mapping.csv"
        predicted_results_path = f"./train_test_data_models/{lang}/{rating_def}/predicted_results/predicted_scores.txt"
        test_file_path = f"./train_test_data_models/{lang}/{rating_def}/test.txt"
        no_ref_user_post_mapping_file_path = f"/home/rohitrr/test/ffm_vp_test/train_test_data_models/{lang}/{rating_def}/user_post_ffm_mapping.csv"
        no_ref_predicted_results_path = f"/home/rohitrr/test/ffm_vp_test/train_test_data_models/{lang}/{rating_def}/predicted_results/predicted_scores.txt"
        no_ref_test_file_path = f"/home/rohitrr/test/ffm_vp_test/train_test_data_models/{lang}/{rating_def}/test.txt"
        
        temp_df = pd.read_csv(user_post_ref_mapping_file_path, delimiter = "\t")
        ref_map = temp_df[~temp_df.feature_name.str.contains("referrer")]
        mapping_df = temp_df[temp_df.feature_name.str.contains("referrer")]
        mapping_df.feature_name = mapping_df.feature_name.apply(lambda name: name[11:])

        no_ref_map = pd.read_csv(no_ref_user_post_mapping_file_path, delimiter = "\t")
        
        if(ref_map.equals(no_ref_map)):
            print("Mappings match so proceeding ....")
        else:
            raise Exception("Mappings don't match")
        
        with open(predicted_results_path) as f:
            lines = f.readlines()
            predicted_scores = [float(score.replace('\n','')) for score in lines]
            
        referrer_mappings = []
        scores = []
        user_mappings = []
        post_mappings = []
        with open(test_file_path) as f:
            for l in f:
                l_split = l.split(':')
                user_mappings.append(int(l_split[1]))
                post_mappings.append(int(l_split[3]))
                referrer_mappings.append(int(l_split[5]))
                scores.append(int(l[0]))
        dataframe_dict = {
            "user_mapping": user_mappings,
            "post_mapping": post_mappings,
            "referrer_mapping": referrer_mappings,
            "ref_score": scores,
            "ref_predicted_score": predicted_scores
            }
        ref_df = pd.DataFrame(dataframe_dict)
        unique_referrer_mappings = ref_df.referrer_mapping.unique()
        
        with open(no_ref_predicted_results_path) as f:
            lines = f.readlines()
            predicted_scores = [float(score.replace('\n','')) for score in lines]

        scores = []
        user_mappings = []
        post_mappings = []
        with open(no_ref_test_file_path) as f:
            for l in f:
                l_split = l.split(':')
                user_mappings.append(int(l_split[1]))
                post_mappings.append(int(l_split[3]))
                scores.append(int(l[0]))
        dataframe_dict = {
            "user_mapping": user_mappings,
            "post_mapping": post_mappings,
            "no_ref_score": scores,
            "no_ref_predicted_score": predicted_scores
            }
        no_ref_df = pd.DataFrame(dataframe_dict)
        df = pd.merge(ref_df, no_ref_df, on=["user_mapping", "post_mapping"])
        print(f"Shape before merging - {ref_df.shape}")
        print(f"Shape after merging - {df.shape}")
        
        ref_auc_scores = []
        no_ref_auc_scores = []
        number_of_ref_samples = []
        langs = []
        rating_defs = []
        for ref_map in unique_referrer_mappings:
            temp_df = df[df.referrer_mapping == ref_map]
            ref_scores = temp_df.ref_score.values
            ref_predicted_scores = temp_df.ref_predicted_score.values
            ref_auc = roc_auc_score(ref_scores, ref_predicted_scores)
            ref_auc_scores.append(ref_auc)
            
            no_ref_scores = temp_df.no_ref_score.values
            no_ref_predicted_scores = temp_df.no_ref_predicted_score.values
            no_ref_auc = roc_auc_score(no_ref_scores, no_ref_predicted_scores)
            no_ref_auc_scores.append(no_ref_auc)
            
            number_of_ref_samples.append(len(scores))
            langs.append(lang)
            rating_defs.append(rating_def)

        referrer_level_auc = {
            "Language": langs,
            "Rating Definition": rating_defs, 
            "mapping": unique_referrer_mappings,
            "referrer_auc": ref_auc_scores,
            "no_referrer_auc": no_ref_auc_scores,
            "number_of_samples": number_of_ref_samples
        }
        
        res_df = pd.DataFrame(referrer_level_auc)
        temp_df = pd.merge(res_df, mapping_df, on=["mapping"])
        if referrer_level_df is None:
            referrer_level_df = temp_df
        else:
            referrer_level_df = referrer_level_df.append(temp_df, ignore_index = True)
        print(f"Completed {lang}-{rating_def}")

Moving to Punjabi-is_vp_succ


/home/rohitrr/miniconda3/envs/py3.8/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Mappings match so proceeding ....
Shape before merging - (92487360, 5)
Shape after merging - (92487358, 7)
Completed Punjabi-is_vp_succ
Moving to Punjabi-is_like


/home/rohitrr/miniconda3/envs/py3.8/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Mappings match so proceeding ....
Shape before merging - (72486564, 5)
Shape after merging - (72486562, 7)
Completed Punjabi-is_like
Moving to Punjabi-is_vp_skip


...

Mappings match so proceeding ....
Shape before merging - (33462835, 5)
Shape after merging - (33462834, 7)
Completed Malayalam-is_like
Moving to Malayalam-is_vp_skip


/home/rohitrr/miniconda3/envs/py3.8/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Mappings match so proceeding ....
Shape before merging - (55346295, 5)
Shape after merging - (55346295, 7)
Completed Malayalam-is_vp_skip
Moving to Malayalam-is_vp_succ2


/home/rohitrr/miniconda3/envs/py3.8/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Mappings match so proceeding ....
Shape before merging - (54673189, 5)
Shape after merging - (54673189, 7)
Completed Malayalam-is_vp_succ2


In [6]:
referrer_level_df.head(5)

,Language,Rating Definition,mapping,referrer_auc,no_referrer_auc,number_of_samples,refferer
0,Punjabi,is_vp_succ,3531454,0.813773,0.812393,92487361,trendingfeed_suggested
1,Punjabi,is_vp_succ,3531451,0.800636,0.797237,92487361,profilepost
2,Punjabi,is_vp_succ,3531452,0.774211,0.773816,92487361,tagfeed
3,Punjabi,is_vp_succ,3531449,0.801454,0.801732,92487361,bucket
4,Punjabi,is_vp_succ,3531450,0.799211,0.800433,92487361,others


In [5]:
referrer_level_df = referrer_level_df.rename(columns = {"feature_name": "refferer"})
referrer_level_df.to_csv("results/referrer_level_metrics_compare.csv", index = False)

### Scrap code

In [47]:
mapping_df = pd.read_csv(user_post_ref_mapping_file_path, delimiter = "\t")
mapping_df = mapping_df[mapping_df.feature_name.str.contains("referrer")]
mapping_df.feature_name = mapping_df.feature_name.apply(lambda name: name[11:])

In [48]:
with open(predicted_results_path) as f:
    lines = f.readlines()
    predicted_scores = [float(score.replace('\n','')) for score in lines]
referrer_mappings = []
scores = []
with open(test_file_path) as f:
    for l in f:
        referrer_mappings.append(int(l.split(':')[5]))
        scores.append(int(l[0]))
dataframe_dict = {
    "referrer_mapping": referrer_mappings,
    "score": scores,
    "predicted_score": predicted_scores
    }
df = pd.DataFrame(dataframe_dict)

In [49]:
unique_referrer_mappings = df.referrer_mapping.unique()
ref_auc_scores = []
number_of_ref_samples = []
for ref_map in unique_referrer_mappings:
    temp_df = df[df.referrer_mapping == ref_map]
    scores = temp_df.score.values
    predicted_scores = temp_df.predicted_score.values
    ref_auc = roc_auc_score(scores, predicted_scores)
    ref_auc_scores.append(ref_auc)
    number_of_ref_samples.append(len(scores))
    
referrer_level_auc = {
    "mapping": unique_referrer_mappings,
    "auc": ref_auc_scores,
    "number_of_samples": number_of_ref_samples
}
res_df = pd.DataFrame(referrer_level_auc)

In [50]:
referrer_level_df = pd.merge(res_df, mapping_df, on=["mapping"])

In [51]:
referrer_level_df

,mapping,auc,number_of_samples,feature_name
0,1974602,0.820723,187308,profilepost
1,1974603,0.800529,404392,tagfeed
2,1974606,0.829898,632503,videofeed
3,1974604,0.843520,1017231,trendingfeed
4,1974605,0.829688,17750572,trendingfeed_suggested
5,1974600,0.820325,1766976,bucket
6,1974607,0.842179,7780777,videofeed_suggested
7,1974601,0.817584,1019904,others


In [4]:
df.head(5)

,feature_name,mapping
0,1_post_1000114392,1
1,1_post_1000320092,2
2,1_post_1000415692,3
3,1_post_1000542292,4
4,1_post_1001177982,5


In [11]:
with open(predicted_scores_path) as f:
    l2 = f.readlines()

In [12]:
l2[0]

'0.0518904\n'

In [19]:
df.head(5)

,referrer_mapping,score,predicted_score
0,1974602,0,0.051890
1,1974602,0,0.029875
2,1974602,0,0.003024
3,1974602,0,0.065021
4,1974602,0,0.006701


In [21]:
unique_referrer_mappings = df.referrer_mapping.unique()

In [37]:
unique_referrer_mappings = df.referrer_mapping.unique()
ref_auc_scores = []
number_of_ref_samples = []
for ref_map in unique_referrer_mappings:
    temp_df = df[df.referrer_mapping == ref_map]
    scores = temp_df.score.values
    predicted_scores = temp_df.predicted_score.values
    ref_auc = roc_auc_score(scores, predicted_scores)
    ref_auc_scores.append(ref_auc)
    number_of_ref_samples.append(len(scores))

In [38]:
referrer_level_auc = {
    "mapping": unique_referrer_mappings,
    "auc": ref_auc_scores,
    "number_of_samples": number_of_ref_samples
}
res_df = pd.DataFrame(referrer_level_auc)

In [72]:
referrer_level_df[["mapping", "feature_name"]]

,mapping,feature_name
0,3531454,trendingfeed_suggested
1,3531451,profilepost
2,3531452,tagfeed
3,3531449,bucket
4,3531450,others
...,...,...
91,2674797,tagfeed
92,2674800,videofeed
93,2674798,trendingfeed
94,2674794,bucket


In [32]:
mapping_df

,feature_name,mapping
1974599,3_referrer_bucket,1974600
1974600,3_referrer_others,1974601
1974601,3_referrer_profilepost,1974602
1974602,3_referrer_tagfeed,1974603
1974603,3_referrer_trendingfeed,1974604
1974604,3_referrer_trendingfeed_suggested,1974605
1974605,3_referrer_videofeed,1974606
1974606,3_referrer_videofeed_suggested,1974607


In [40]:
pd.merge(res_df, mapping_df, on=["mapping"])

,mapping,auc,number_of_samples,feature_name
0,1974602,0.820723,187308,3_referrer_profilepost
1,1974603,0.800529,404392,3_referrer_tagfeed
2,1974606,0.829898,632503,3_referrer_videofeed
3,1974604,0.843520,1017231,3_referrer_trendingfeed
4,1974605,0.829688,17750572,3_referrer_trendingfeed_suggested
5,1974600,0.820325,1766976,3_referrer_bucket
6,1974607,0.842179,7780777,3_referrer_videofeed_suggested
7,1974601,0.817584,1019904,3_referrer_others


In [146]:
referrer_level_df = None
for lang in LANGS:
    for _, rating_def in rating_def_dict.items():
        print(f"Moving to {lang}-{rating_def}")
        user_post_ref_mapping_file_path = f"./train_test_data_models/{lang}/{rating_def}/user_post_ffm_mapping.csv"
        predicted_results_path = f"./train_test_data_models/{lang}/{rating_def}/predicted_results/predicted_scores.txt"
        test_file_path = f"./train_test_data_models/{lang}/{rating_def}/test.txt"
        
        mapping_df = pd.read_csv(user_post_ref_mapping_file_path, delimiter = "\t")
        mapping_df = mapping_df[mapping_df.feature_name.str.contains("referrer")]
        mapping_df.feature_name = mapping_df.feature_name.apply(lambda name: name[11:])
        
        with open(predicted_results_path) as f:
            lines = f.readlines()
            predicted_scores = [float(score.replace('\n','')) for score in lines]
        referrer_mappings = []
        scores = []
        with open(test_file_path) as f:
            for l in f:
                referrer_mappings.append(int(l.split(':')[5]))
                scores.append(int(l[0]))
        dataframe_dict = {
            "referrer_mapping": referrer_mappings,
            "score": scores,
            "predicted_score": predicted_scores
            }
        df = pd.DataFrame(dataframe_dict)
        unique_referrer_mappings = df.referrer_mapping.unique()
        ref_auc_scores = []
        number_of_ref_samples = []
        langs = []
        rating_defs = []
        for ref_map in unique_referrer_mappings:
            temp_df = df[df.referrer_mapping == ref_map]
            scores = temp_df.score.values
            predicted_scores = temp_df.predicted_score.values
            ref_auc = roc_auc_score(scores, predicted_scores)
            ref_auc_scores.append(ref_auc)
            number_of_ref_samples.append(len(scores))
            langs.append(lang)
            rating_defs.append(rating_def)

        referrer_level_auc = {
            "Language": langs,
            "Rating Definition": rating_defs, 
            "mapping": unique_referrer_mappings,
            "auc": ref_auc_scores,
            "number_of_samples": number_of_ref_samples
        }
        res_df = pd.DataFrame(referrer_level_auc)
        temp_df = pd.merge(res_df, mapping_df, on=["mapping"])
        if referrer_level_df is None:
            referrer_level_df = temp_df
        else:
            referrer_level_df = referrer_level_df.append(temp_df, ignore_index = True)
        print(f"Completed {lang}-{rating_def}")

Moving to Punjabi-is_vp_succ


KeyboardInterrupt: 

In [152]:
referrer_level_df = None
for lang in LANGS:
    for _, rating_def in rating_def_dict.items():
        print(f"Moving to {lang}-{rating_def}")
        user_post_ref_mapping_file_path = f"./train_test_data_models/{lang}/{rating_def}/user_post_ffm_mapping.csv"
        predicted_results_path = f"./train_test_data_models/{lang}/{rating_def}/predicted_results/predicted_scores.txt"
        test_file_path = f"./train_test_data_models/{lang}/{rating_def}/test.txt"
        no_ref_user_post_mapping_file_path = f"/home/rohitrr/test/ffm_vp_test/train_test_data_models/{lang}/{rating_def}/user_post_ffm_mapping.csv"
        no_ref_predicted_results_path = f"/home/rohitrr/test/ffm_vp_test/train_test_data_models/{lang}/{rating_def}/predicted_results/predicted_scores.txt"
        no_ref_test_file_path = f"/home/rohitrr/test/ffm_vp_test/train_test_data_models/{lang}/{rating_def}/test.txt"
        
        temp_df = pd.read_csv(user_post_ref_mapping_file_path, delimiter = "\t")
        ref_map = temp_df[~temp_df.feature_name.str.contains("referrer")]
        mapping_df = temp_df[temp_df.feature_name.str.contains("referrer")]
        mapping_df.feature_name = mapping_df.feature_name.apply(lambda name: name[11:])

        no_ref_map = pd.read_csv(no_ref_user_post_mapping_file_path, delimiter = "\t")
        
        if(ref_map.equals(no_ref_map)):
            print("Mappings match so proceeding ....")
        else:
            raise Exception("Mappings don't match")
        
        with open(predicted_results_path) as f:
            lines = f.readlines()
            predicted_scores = [float(score.replace('\n','')) for score in lines]
            
        referrer_mappings = []
        scores = []
        user_mappings = []
        post_mappings = []
        with open(test_file_path) as f:
            for l in f:
                l_split = l.split(':')
                user_mappings.append(int(l_split[1]))
                post_mappings.append(int(l_split[3]))
                referrer_mappings.append(int(l_split[5]))
                scores.append(int(l[0]))
        dataframe_dict = {
            "user_mapping": user_mappings,
            "post_mapping": post_mappings,
            "referrer_mapping": referrer_mappings,
            "ref_score": scores,
            "ref_predicted_score": predicted_scores
            }
        ref_df = pd.DataFrame(dataframe_dict)
        unique_referrer_mappings = ref_df.referrer_mapping.unique()
        
        with open(no_ref_predicted_results_path) as f:
            lines = f.readlines()
            predicted_scores = [float(score.replace('\n','')) for score in lines]

        scores = []
        user_mappings = []
        post_mappings = []
        with open(no_ref_test_file_path) as f:
            for l in f:
                l_split = l.split(':')
                user_mappings.append(int(l_split[1]))
                post_mappings.append(int(l_split[3]))
                scores.append(int(l[0]))
        dataframe_dict = {
            "user_mapping": user_mappings,
            "post_mapping": post_mappings,
            "no_ref_score": scores,
            "no_ref_predicted_score": predicted_scores
            }
        no_ref_df = pd.DataFrame(dataframe_dict)
        df = pd.merge(ref_df, no_ref_df, on=["user_mapping", "post_mapping"])
        print(f"Shape before merging - {ref_df.shape}")
        print(f"Shape after merging - {df.shape}")
        
        ref_auc_scores = []
        no_ref_auc_scores = []
        number_of_ref_samples = []
        langs = []
        rating_defs = []
        for ref_map in unique_referrer_mappings:
            temp_df = df[df.referrer_mapping == ref_map]
            ref_scores = temp_df.ref_score.values
            ref_predicted_scores = temp_df.ref_predicted_score.values
            ref_auc = roc_auc_score(ref_scores, ref_predicted_scores)
            ref_auc_scores.append(ref_auc)
            
            no_ref_scores = temp_df.no_ref_score.values
            no_ref_predicted_scores = temp_df.no_ref_predicted_score.values
            no_ref_auc = roc_auc_score(no_ref_scores, no_ref_predicted_scores)
            no_ref_auc_scores.append(no_ref_auc)
            
            number_of_ref_samples.append(len(scores))
            langs.append(lang)
            rating_defs.append(rating_def)

        referrer_level_auc = {
            "Language": langs,
            "Rating Definition": rating_defs, 
            "mapping": unique_referrer_mappings,
            "referrer_auc": ref_auc_scores,
            "no_referrer_auc": no_ref_auc_scores,
            "number_of_samples": number_of_ref_samples
        }
        
        res_df = pd.DataFrame(referrer_level_auc)
        temp_df = pd.merge(res_df, mapping_df, on=["mapping"])
        if referrer_level_df is None:
            referrer_level_df = temp_df
        else:
            referrer_level_df = referrer_level_df.append(temp_df, ignore_index = True)
        print(f"Completed {lang}-{rating_def}")

Moving to Punjabi-is_vp_succ
Mappings match so proceeding ....
Shape before merging - (92487360, 5)
Shape after merging - (92487358, 7)
Completed Punjabi-is_vp_succ


In [155]:
referrer_level_df

,Language,Rating Definition,mapping,referrer_auc,no_referrer_auc,number_of_samples,feature_name
0,Punjabi,is_vp_succ,3531454,0.813773,0.813773,92487361,trendingfeed_suggested
1,Punjabi,is_vp_succ,3531451,0.800636,0.800636,92487361,profilepost
2,Punjabi,is_vp_succ,3531452,0.774211,0.774211,92487361,tagfeed
3,Punjabi,is_vp_succ,3531449,0.801454,0.801454,92487361,bucket
4,Punjabi,is_vp_succ,3531450,0.799211,0.799211,92487361,others
5,Punjabi,is_vp_succ,3531455,0.802020,0.802020,92487361,videofeed
6,Punjabi,is_vp_succ,3531453,0.834032,0.834032,92487361,trendingfeed
7,Punjabi,is_vp_succ,3531456,0.812813,0.812813,92487361,videofeed_suggested
8,Punjabi,is_vp_succ,3531454,0.813773,0.812393,92487361,trendingfeed_suggested
9,Punjabi,is_vp_succ,3531451,0.800636,0.797237,92487361,profilepost


In [154]:
ref_auc_scores = []
no_ref_auc_scores = []
number_of_ref_samples = []
langs = []
rating_defs = []
for ref_map in unique_referrer_mappings:
    temp_df = df[df.referrer_mapping == ref_map]
    ref_scores = temp_df.ref_score.values
    ref_predicted_scores = temp_df.ref_predicted_score.values
    ref_auc = roc_auc_score(ref_scores, ref_predicted_scores)
    ref_auc_scores.append(ref_auc)

    no_ref_scores = temp_df.no_ref_score.values
    no_ref_predicted_scores = temp_df.no_ref_predicted_score.values
    no_ref_auc = roc_auc_score(no_ref_scores, no_ref_predicted_scores)
    no_ref_auc_scores.append(no_ref_auc)

    number_of_ref_samples.append(len(ref_scores))
    langs.append(lang)
    rating_defs.append(rating_def)

referrer_level_auc = {
    "Language": langs,
    "Rating Definition": rating_defs, 
    "mapping": unique_referrer_mappings,
    "referrer_auc": ref_auc_scores,
    "no_referrer_auc": no_ref_auc_scores,
    "number_of_samples": number_of_ref_samples
}

res_df = pd.DataFrame(referrer_level_auc)
temp_df = pd.merge(res_df, mapping_df, on=["mapping"])
if referrer_level_df is None:
    referrer_level_df = temp_df
else:
    referrer_level_df = referrer_level_df.append(temp_df, ignore_index = True)
print(f"Completed {lang}-{rating_def}")

Completed Punjabi-is_vp_succ


In [92]:
test_path = "/home/rohitrr/test/ffm_vp_test/train_test_data_models/Bengali/is_vp_succ/test.txt"
user_mappings = []
post_mappings = []
referrer_mappings = []
with open(test_path) as f:
    for l in f:
        l_split = l.split(':')
        user_mappings.append(int(l_split[1]))
        post_mappings.append(int(l_split[3]))
        
temp_dict = {
    "user_mapping": user_mappings,
    "post_mapping": post_mappings
}
temp_df = pd.DataFrame(temp_dict)

In [93]:
temp_df.head(5)

,user_mapping,post_mapping
0,118412,20560
1,709563,15929
2,461262,34596
3,930551,10630
4,1904296,4485


In [95]:
no_ref_df = temp_df

In [96]:
ref_df.shape, no_ref_df.shape

((30559663, 3), (30559663, 2))

In [97]:
t_df_1 = pd.merge(ref_df, no_ref_df, on=["user_mapping", "post_mapping"])

In [98]:
t_df_1.shape

(30559661, 3)

In [104]:
t_df_1.tail(5)

,user_mapping,post_mapping,referrer_mapping
30559656,427855,18059,1974607
30559657,99226,16688,1974607
30559658,104875,1718,1974607
30559659,619200,16688,1974607
30559660,318646,14270,1974607


In [105]:
ref_df.tail(5)

,user_mapping,post_mapping,referrer_mapping
30559658,427855,18059,1974607
30559659,99226,16688,1974607
30559660,104875,1718,1974607
30559661,619200,16688,1974607
30559662,318646,14270,1974607


In [127]:
uv = t_df_1.user_mapping.values
uv_1 = ref_df.user_mapping.values
pv = t_df_1.post_mapping.values
pv_1 = ref_df.post_mapping.values

In [110]:
t_df_2 = ref_df[ref_df.user_mapping.isin(uv)]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [123]:
temp_set = set(uv).difference(uv_1)

In [124]:
len(temp_set)

0

In [150]:
len(predicted_scores), len(scores)

(92487360, 92487360)

In [125]:
temp_set

set()

In [126]:
ref_df[ref_df.user_mapping == 1255122]

,user_mapping,post_mapping,referrer_mapping
10545509,1255122,28790,1974602


In [128]:
set(list(zip(uv, pv)))^set(list(zip(uv_1, pv_1)))

{(221449, 28790), (1255122, 28790)}

In [133]:
user_post_ref_mapping_file_path = "./train_test_data_models/Bengali/is_vp_succ/user_post_ffm_mapping.csv"
ref_map = pd.read_csv(user_post_ref_mapping_file_path, delimiter="\t")
user_post_ref_mapping_file_path = "/home/rohitrr/test/ffm_vp_test/train_test_data_models/Bengali/is_vp_succ/user_post_ffm_mapping.csv"
no_ref_map = pd.read_csv(user_post_ref_mapping_file_path, delimiter="\t")

In [136]:
ref_map = ref_map[~ref_map.feature_name.str.contains("referrer")]

In [138]:
ref_map.shape, no_ref_map.shape

((1974599, 2), (1974599, 2))

In [139]:
ref_map.equals(no_ref_map)

True

In [143]:
ref_map.head(5)

,feature_name,mapping
2674793,bucket,2674794
2674794,others,2674795
2674795,profilepost,2674796
2674796,tagfeed,2674797
2674797,trendingfeed,2674798
